# Clear Cache Methods

Here are various ways to clear different types of cache in your notebook environment:

In [ ]:
# 1. RESTART KERNEL (Most Effective)
# Use: Kernel -> Restart Kernel in VS Code menu
# This clears ALL variables, imports, and function definitions

# 2. Clear all variables manually
%reset -f

# 3. Clear specific modules from cache
import sys
modules_to_clear = [mod for mod in sys.modules.keys() if 'doorloop' in mod.lower() or 'connecteam' in mod.lower()]
for mod in modules_to_clear:
    if mod in sys.modules:
        del sys.modules[mod]
        print(f"Cleared module: {mod}")

print("✅ Manual cache clearing completed!")

In [ ]:
# 4. Clear Python import cache
import importlib
import sys

# Force reload of your custom modules
def force_reload_modules():
    """Force reload all custom modules"""
    custom_modules = [
        'doorloop_mcp_server',
        'connectteam_mcp_server', 
        'doorloop_direct',
        'doorloop_services',
        'connecteam_service'
    ]
    
    for module_name in custom_modules:
        if module_name in sys.modules:
            print(f"Reloading: {module_name}")
            importlib.reload(sys.modules[module_name])
        else:
            print(f"Module not loaded: {module_name}")

# Run the reload
force_reload_modules()

print("✅ Module reload completed!")

In [ ]:
# 5. Clear environment variables cache
import os
from dotenv import load_dotenv

# Force reload .env file
load_dotenv(override=True)  # override=True forces reload even if already loaded

# Verify environment variables are loaded fresh
print("🔑 Current API Keys Status:")
print(f"DOORLOOP_API_KEY: {'✅ Set' if os.getenv('DOORLOOP_API_KEY') else '❌ Missing'}")
print(f"CONNECTTEAM_API_KEY: {'✅ Set' if os.getenv('CONNECTTEAM_API_KEY') else '❌ Missing'}")
print(f"CONNECTEAM_TASKBOARD_ID: {'✅ Set' if os.getenv('CONNECTEAM_TASKBOARD_ID') else '❌ Missing'}")

print("✅ Environment variables reloaded!")

## **Quick Fix for Your Current Issue**

If your `list_tasks` function isn't working, try these steps in order:

In [ ]:
# IMMEDIATE FIX: Reload and test your list_tasks function
from dotenv import load_dotenv
import os

# Force reload environment variables
load_dotenv(override=True)

# Test with explicit taskboard_id parameter
def test_list_tasks():
    """Test the list_tasks function with proper parameters"""
    
    # Check if taskboard_id is in environment
    taskboard_id = os.getenv("CONNECTEAM_TASKBOARD_ID")
    
    if not taskboard_id:
        print("❌ CONNECTEAM_TASKBOARD_ID not found in environment!")
        print("💡 Add it to your .env file or pass it as parameter")
        return
    
    print(f"✅ Using taskboard_id: {taskboard_id}")
    
    # Call your function with the taskboard_id
    result = list_tasks("all", 10, 0, taskboard_id)
    print("📊 Result:", result)
    return result

# Run the test
test_result = test_list_tasks()

In [17]:
import os
import sys
import json
import requests
from dotenv import load_dotenv

# Load environment variables from a .env file so API keys are available
load_dotenv()

# Optional import of FastMCP: handle missing package gracefully in the notebook
try:
	from mcp.server.fastmcp import FastMCP
except Exception:
	FastMCP = None

In [6]:
def generate_report():
    """Fetch DoorLoop profit-and-loss (safe, debuggable)."""
    import os, requests
    api_key = os.getenv("DOORLOOP_API_KEY")
    if not api_key:
        return {"error": "DOORLOOP_API_KEY not found in environment variables"}

    #base_url = os.getenv("DOORLOOP_API_BASE", "https://app.doorloop.com")
    base_url = "https://app.doorloop.com"
    # NOTE: verify the exact report path and query parameter names with DoorLoop docs
    endpoint = f"{base_url.rstrip('/')}/api/reports/balance-sheet-summary?filter_accountingMethod=CASH"
    headers = {"Authorization": f"Bearer {api_key}", "accept": "application/json"}

    try:
        resp = requests.get(endpoint, headers=headers, timeout=15)
    except requests.exceptions.RequestException as exc:
        return {"error": "Request failed", "exception": str(exc)}

    # Basic debug info
    result = {"status": resp.status_code, "content_type": resp.headers.get("Content-Type", "")}

    # Try parse JSON
    try:
        j = resp.json()
    except ValueError:
        # response not JSON
        result["body"] = resp.text[:2000]
        return result

    # If status not OK, return JSON body for debugging
    if not resp.ok:
        result["body_json"] = j
        return result

    # Try to normalize with pandas if available
    try:
        import pandas as pd
    except ModuleNotFoundError:
        # return raw JSON when pandas not installed
        return {"result": j, "note": "pandas not installed; install it to normalize"}

    # Choose the data payload we'll normalize (try likely keys)
    data = None
    if isinstance(j, dict):
        for candidate in ("rent_roll", "data", "items", "results", "tenants", "report", "rows"):
            if candidate in j and isinstance(j[candidate], (list, dict)):
                data = j[candidate]
                break
        if data is None:
            # pick first list value if any
            for v in j.values():
                if isinstance(v, list):
                    data = v
                    break
    else:
        data = j

    # Normalize safely
    try:
        if data is None:
            df = pd.DataFrame()  # empty
        else:
            df = pd.json_normalize(data)
            df = df.rename(columns={"undepositedFundsSplits.ePaymentsTotal":"undeposited ePayments",
                                    "undepositedFundsSplits.manualEntriesTotal":"undeposited manualEntries"})
            print(df.columns)
            # Fix: fillna needs to be assigned back or use inplace=True
            df = df.fillna(0)  # Returns new DataFrame with NaN replaced by 0
            # Alternative: df.fillna(0, inplace=True)  # Modifies original DataFrame
        return df#{"status": resp.status_code, "result": df.to_dict(orient="records")}
    except Exception as exc:
        return {"error": "Normalization failed", "exception": str(exc), "raw": j}

In [93]:
import textwrap
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

class DoorLoopReportGenerator:
    """Professional PDF report generator for DoorLoop data with text wrapping and customizable styling."""
    
    def __init__(self, 
                 col_width=5.0,  # Even wider columns
                 row_height=0.9,  # Taller rows for wrapped text
                 font_size=11,  # Larger font size
                 header_font_size=12,
                 max_text_width=40):  # More characters before wrapping
        self.col_width = col_width
        self.row_height = row_height
        self.font_size = font_size
        self.header_font_size = header_font_size
        self.max_text_width = max_text_width
        
    def wrap_text(self, text, width=None):
        """Wrap text to fit within column width."""
        if width is None:
            width = self.max_text_width
            
        if not isinstance(text, str):
            text = str(text)
            
        if len(text) <= width:
            return text
            
        # Use textwrap for clean line breaks
        wrapped_lines = textwrap.wrap(text, width=width, break_long_words=False)
        return '\n'.join(wrapped_lines[:3])  # Limit to 3 lines max
    
    def format_numeric_value(self, value):
        """Format numeric values with proper comma separation - ensure no truncation."""
        if pd.isna(value) or value is None:
            return ""
        elif isinstance(value, (int, float)):
            if value == 0:
                return "0"
            elif isinstance(value, float):
                # For very large numbers, use scientific notation to prevent cutting
                if abs(value) >= 1000000:
                    return f"{value:,.0f}"  # No decimals for large numbers
                else:
                    return f"{value:,.2f}"
            else:
                return f"{value:,}"
        else:
            # Convert to string and ensure it's not too long
            str_val = str(value)
            if len(str_val) > 15:  # If text is very long, wrap it
                return self.wrap_text(str_val, 15)
            return str_val
    
    def prepare_table_data(self, df):
        """Prepare and format table data with text wrapping."""
        table_data = []
        
        for _, row in df.iterrows():
            formatted_row = []
            for i, cell in enumerate(row):
                # Format ALL cells properly to prevent cutting
                if isinstance(cell, (int, float)) and not pd.isna(cell):
                    # Always use numeric formatting for numbers
                    formatted_cell = self.format_numeric_value(cell)
                else:
                    # For text, wrap if needed
                    if isinstance(cell, str) and len(cell) > self.max_text_width:
                        formatted_cell = self.wrap_text(cell, self.max_text_width)
                    else:
                        formatted_cell = self.wrap_text(str(cell), self.max_text_width)
                
                formatted_row.append(formatted_cell)
            table_data.append(formatted_row)
        
        return table_data
    
    def prepare_column_headers(self, df):
        """Prepare column headers with wrapping."""
        headers = []
        for col in df.columns:
            # Clean up column names
            clean_col = str(col).replace('_', ' ').title()
            # Wrap long headers
            wrapped_header = self.wrap_text(clean_col, 25)  # More space for headers
            headers.append(wrapped_header)
        return headers
    
    def calculate_column_widths(self, df):
        """Calculate UNIFORM column widths - all columns same width but WIDER."""
        num_cols = len(df.columns)
        
        # Make all columns even wider to prevent any cutting
        uniform_width = 0.28  # Increased from 0.22 to 0.28 for more space
        col_widths = [uniform_width] * num_cols  # Create list with same width for all columns
        
        return col_widths
    
    def style_table(self, table, df, col_widths):
        """Apply styling to the matplotlib table."""
        num_rows, num_cols = len(df), len(df.columns)
        
        # Apply column widths and row heights
        for i in range(num_cols):
            for j in range(num_rows + 1):  # +1 for header
                cell = table[(j, i)]
                cell.set_width(col_widths[i])  # All columns now have same width
                cell.set_height(0.15)  # Even taller rows for better spacing
                
                # Add padding to prevent text cutting
                cell.PAD = 0.05  # More internal padding
                
                # Header row styling
                if j == 0:
                    cell.set_facecolor('#2E75B6')
                    cell.set_text_props(
                        weight='bold', 
                        color='white', 
                        fontsize=self.header_font_size,
                        verticalalignment='center',
                        ha='center'  # Center align headers for uniform look
                    )
                else:
                    # Data row styling
                    if j % 2 == 0:
                        cell.set_facecolor('#F8F9FA')
                    else:
                        cell.set_facecolor('white')
                    
                    # Set text properties for data cells - all left aligned for consistency
                    cell.set_text_props(
                        ha='left',  # All columns left-aligned for uniform appearance
                        fontsize=self.font_size,
                        verticalalignment='center'
                    )
        
        return table
    
    def generate_pdf(self, df, filename="Report.pdf", title="Nest Host Financial Report"):
        """Generate a professional PDF report from DataFrame."""
        if not hasattr(df, 'values') or not hasattr(df, 'columns'):
            raise ValueError("Input must be a pandas DataFrame")
        
        num_cols = len(df.columns)
        num_rows = len(df)
        
        # Calculate figure dimensions with wider layout to prevent cutting
        fig_width = max(28, num_cols * self.col_width)  # Much wider base (was 24)
        fig_height = max(18, num_rows * self.row_height + 7)  # Taller figure (was 16)
        
        with PdfPages(filename) as pdf:
            fig = plt.figure(figsize=(fig_width, fig_height))
            ax = fig.add_subplot(111)
            ax.axis('off')
            
            # Prepare data
            table_data = self.prepare_table_data(df)
            headers = self.prepare_column_headers(df)
            col_widths = self.calculate_column_widths(df)
            
            # Create table
            table = ax.table(
                cellText=table_data,
                colLabels=headers,
                cellLoc='left',
                loc='center',
                bbox=[0, 0, 1, 0.72]  # More space for title and margins
            )
            
            # Style the table
            table.auto_set_font_size(False)
            self.style_table(table, df, col_widths)
            
            # Add title with Nest Host company branding
            fig.suptitle(title, fontsize=26, fontweight='bold', y=0.95)
            
            # Add company name and subtitle
            plt.figtext(0.5, 0.88, 
                       f'Nest Host | Generated on {pd.Timestamp.now().strftime("%B %d, %Y")} | {num_rows} entries', 
                       ha='center', fontsize=18, style='italic')
            
            # Save with high quality
            pdf.savefig(fig, bbox_inches='tight', dpi=300, 
                       facecolor='white', edgecolor='none')
            plt.close(fig)
        
        return {
            "filename": filename,
            "rows": num_rows,
            "columns": num_cols,
            "figure_size": f"{fig_width:.1f}\" × {fig_height:.1f}\"",
            "status": "success"
        }

# Usage with extra wide columns to prevent cutting

    

In [ ]:
# result = generate_report()

if hasattr(result, 'values') and hasattr(result, 'columns'):
    # Create report generator instance with extra wide columns
    generator = DoorLoopReportGenerator(
        col_width=4.5,           # Even wider columns
        row_height=1.0,          # Taller rows for better spacing
        font_size=11,            # Larger body text size
        header_font_size=12,     # Larger header text size
        max_text_width=40        # More characters before text wrapping
    )
    
    # Generate the PDF with Nest Host branding
    report_info = generator.generate_pdf(
        df=result, 
        filename="DoorLoop_Balance_Sheet.pdf",
        title="Nest Host - DoorLoop Balance Sheet Report"
    )

In [ ]:
def retrieve_doorloop_communication():
    """Retrieve tenant data from the DoorLoop API"""
    api_key = os.getenv("DOORLOOP_API_KEY")
    if not api_key:
        return {"error": "DOORLOOP_API_KEY not found in environment variables"}

    base_url = os.getenv("DOORLOOP_API_BASE", "https://app.doorloop.com")
    endpoint =f"{base_url.rstrip('/')}/api/communications"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "accept": "application/json",
        "content-type": "application/json"
    }
    try: # If the response is JSON return that, otherwise include a short text body for debugging
        response = requests.get(endpoint, headers=headers, timeout=10)
        content_type = response.headers.get("Content-Type", "")
        if response.ok:
            if "application/json" in content_type:
                return response.json()  # Try to parse JSON even if header is missing
            try:
                return response.json()
            except Exception:
                return {"status": response.status_code, "body": response.text[:1000]}
        else:
            try:
                resp_json = response.json()
            except Exception:
                resp_json = None
            return {
                "error": "Failed to fetch tenants",
                "status": response.status_code,
                "response": resp_json,
            }
    except requests.exceptions.RequestException as exc:
        return {"error": "Request failed", "exception": str(exc)}

In [50]:
def generate_conversation(limit: int, offset: int):
    """Fetch connecteam taskboard (safe, debuggable)."""
    import os, requests  # Import os and requests modules
    from dotenv import load_dotenv
    load_dotenv()
    
    # Use the exact variable name from .env file: CONNECTTEAM_API_KEY (with double 'T')
    api_key = os.getenv("CONNECTTEAM_API_KEY")
    if not api_key:
        return {"error": "CONNECTTEAM_API_KEY not found in environment variables"}

    base_url = os.getenv("CONNECTEAM_API_BASE", "https://app.connecteam.com")
    # NOTE: verify the exact report path and query parameter names with Connecteam docs
    endpoint = f"{base_url.rstrip('/')}/tasks/v1/taskboards"
    params = {"limit": limit, "offset": offset}
    # Try different authorization header formats for Connecteam API
    headers = {
        "x-api-key": api_key,  # Many APIs use x-api-key instead of Authorization Bearer
        "accept": "application/json",
        "content-type": "application/json"
    }

    try: # If the response is JSON return that, otherwise include a short text body for debugging
        response = requests.get(endpoint, headers=headers,params=params, timeout=10)
        content_type = response.headers.get("Content-Type", "")
        if response.ok:
            if "application/json" in content_type:
                return response.json()  # Try to parse JSON even if header is missing
            try:
                return response.json()
            except Exception:
                return {"status": response.status_code, "body": response.text[:1000]}
        else:
            return {
                "error": "Failed to fetch tenants",
                "status": response.status_code,
                "response": response.json(),
            }
    except requests.exceptions.RequestException as exc:
        return {"error": "Request failed", "exception": str(exc)}


In [ ]:
# Test the updated function with correct variable name
result = generate_conversation(10,0)
print("Result:", result)
print("Type:", type(result))

In [54]:

def list_tasks(status: str , limit: int, offset:int, taskboard_id: str | None = None):
    import os, requests  # Import os and requests modules
    from dotenv import load_dotenv
    load_dotenv()
    """List tasks with simple pagination and status filter."""
    api_key = os.getenv("CONNECTTEAM_API_KEY")
    if not api_key:
        return {"error": "CONNECTTEAM_API_KEY not found in environment variables"}

    # Use provided taskboard_id or fallback to environment variable
    if not taskboard_id:
        taskboard_id = os.getenv("CONNECTEAM_TASKBOARD_ID")
    
    if not taskboard_id:
        return {"error": "taskboard_id is required. Provide it as parameter or set CONNECTEAM_TASKBOARD_ID environment variable"}

    base_url = os.getenv("CONNECTTEAM_API_BASE", "https://api.connecteam.com")
    # Correct URL format: https://api.connecteam.com/tasks/v1/taskboards/taskBoardId/tasks?status=all&limit=10&offset=0
    endpoint = f"{base_url.rstrip('/')}/tasks/v1/taskboards/{taskboard_id}/tasks"
    params = {"status": status, "limit": limit, "offset": offset}
    headers = {"x-api-key": api_key, "accept": "application/json"}
    try:
        resp = requests.get(endpoint, headers=headers, params=params, timeout=10)
    except requests.exceptions.RequestException as exc:
        return {"error": "Request failed", "exception": str(exc)}

    try:
        return resp.json()
    except Exception:
        return {"status": resp.status_code, "body_snippet": resp.text[:1000]}



In [ ]:
taskboard_id = os.getenv("CONNECTEAM_TASKBOARD_ID")
list_tasks("all",10,0)

In [ ]:
import os
import sys
import json
import requests
from dotenv import load_dotenv




## inorder to create we must create a customer object, with 
### cust_id, email, displayname, display company

In [ ]:
import hashlib

password = "Goldyg_77"
encoded_object = hashlib.sha256(password.encode("utf-8"))
hashed_obj = encoded_object.hexdigest()


print(hashed_obj)


39838c27ce0aec14f6ff49a71f69e75842678f18f62ee50782cc7f588e3f8cd7


In [1]:
import logging
import sys
from pathlib import Path

In [ ]:
PROJECT_ROOT = Path(__file__).absolute().parent.parent

if str(PROJECT_ROOT) not in sys.path:
 	sys.path.insert(0, str(PROJECT_ROOT))
  
try:
	# Import after adjusting sys.path
	from mcp_server import doorloop_mcp_server
except Exception as exc:
	raise ImportError(f"Failed to import mcp_server. Ensure project root is correct: {PROJECT_ROOT}\nOriginal error: {exc}")

In [ ]:
def rawdata_extraction(raw_data):
    initial_data = raw_data['data']
    return initial_data

file = "C://Users//bhasi//OneDrive//Desktop//chatbot//Microservices-Backend//raw_data.json"
print(rawdata_extraction(file))